### Reference url

In [29]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

In [639]:
from PIL import Image 
import os 
import PIL 
import glob
import time

In [385]:
def extract_int(sample):
    list = re.findall(r'\d+', sample)
    while True:
        try:
            res = int("".join(map(str, list)))
            break
        except ValueError:
            res = np.nan
            break
    return res

In [124]:
url = 'https://www.autoscout24.fr/offres/ducati-xdiavel-essence-rouge-c9a6b7d5-939e-4be8-a45a-bc8adf43075c?cldtidx=7&cldtsrc=listPage&searchId=-842076858'

In [125]:
response = requests.get(url)
soup_ = BeautifulSoup(response.content, "html.parser")

In [348]:
def as24_announce_reference(soup_):
    guid = soup_.find("div", class_="sc-pull-right cldt-buttons cldt-hide-on-print cldt-version-one")
    guid = guid.find("a", class_="btn-watchlist").attrs['data-classified-guid']
    return guid

### Structure du DataFrame

#### Template

In [30]:
# template Aloys
announce_template = pd.DataFrame({"url": "",
                                  "reference": [np.nan],
                                  "model": "",
                                  "price": [np.nan],
                                  "mileage": [0],
                                  "power_kW": [0],
                                  "power_CH": [0],
                                  "État": "", 
                                  "Propriétaires préc.": [np.nan],
                                  "Dernier contrôle technique": [datetime.datetime(1970, 1, 1)],
                                  "Garantie": [datetime.datetime(1970, 1, 1)],
                                  "Carnet d'entretien": [np.nan],
                                  "Marque": "", 
                                  "N° d'annonce": [np.nan], 
                                  "Année": [datetime.datetime(1970, 1, 1)], 
                                  "Couleur extérieure": "", 
                                  "Type de peinture": "", 
                                  "Couleur originale": "", 
                                  "Carrosserie": "",
                                  "Transmission": "", 
                                  "Vitesses": np.nan, 
                                  "Cylindrée": np.nan, 
                                  "Cylindres": [0], 
                                  "Poids à vide" : np.nan
                                 })
announce_template

,url,searchId,title,price,mileage,power_kW,power_CH,État,Propriétaires préc.,Dernier contrôle technique,...,Année,Couleur extérieure,Type de peinture,Couleur originale,Carrosserie,Transmission,Vitesses,Cylindrée,Cylindres,Poids à vide
0,,NaN,,NaN,0,0,0,,NaN,1970-01-01,...,1970-01-01,,,,,,NaN,NaN,0,NaN


### Extraction

#### Caractéristiques premières 

##### get images

In [172]:
def save_images(images_list, uniq_id):
    k = 0
    
    for image_url in image_list[0:3]:
        img_data = requests.get(image_url).content
        with open(f'images/as24-{uniq_id}-{k}.jpg', 'wb') as handler:
            handler.write(img_data)
            image = Image.open(f'images/as24-{uniq_id}-{k}.jpg') 
            ratio = image.size[0] / image.size[1]
            image = image.resize((300,int(300/ratio)))
            image.save(f'images/as24-{uniq_id}-{k}.jpg',optimize = True, quality = 50)
        k+=1

In [173]:
def get_images(soup_, uniq_id):
    container = soup_.find_all('div',class_="gallery-picture")

    images_list = []

    for image in container:
        images_list.append(image.find("img").attrs['data-fullscreen-src'])
    
    save_images(images_list, uniq_id)
    return

In [174]:
uniq_id = "as24_" + get_announce_reference(soup_)

get_images(soup_, uniq_id)

In [84]:
container = soup_.find_all('div',class_="gallery-picture")

images_list = []

for image in container:
    images_list.append(image.find("img").attrs['data-fullscreen-src'])

images_list

['https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_75407af6-c73c-40f1-9965-f7dacad8f559.jpg/1280x960.jpg',
 'https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_e536507a-07b5-4ef0-947c-67f18fb1f01c.jpg/1280x960.jpg',
 'https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_2b0cbd52-c8b0-4cd8-8f0b-541d8850ccde.jpg/1280x960.jpg',
 'https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_46a7a692-1ea9-463a-9b5d-f363fa7235de.jpg/1280x960.jpg',
 'https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_831e743c-6f96-4689-bc71-50978ac46982.jpg/1280x960.jpg',
 'https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_c90b25cc-4165-4733-ae3e-cd9e8662fa0b.jpg/1280x960.jpg',
 'https://prod.pictures.autoscout24.net/listing-images/d617ea01-66e8-49ff-9817-b7ea54d90d63_2daa17c6-0cde-4850-a

##### Titre

In [744]:
url = 'https://www.autoscout24.fr/offres/others-others-climax-m1-bobber-essence-noir-082034e4-655e-4e03-863e-734200a01876'
response = requests.get(url)
soup_ = BeautifulSoup(response.content, "html.parser")

In [745]:
title = soup_.find("div", class_="cldt-headline").find("span", class_="cldt-detail-makemodel sc-ellipsis").text
title_version = soup_.find("div", class_="cldt-headline").find("span", class_="cldt-detail-version sc-ellipsis").text

In [746]:
title, title_version

('Autres', 'Climax M1 Bobber')

##### Prix

In [6]:
price = extract_int(soup_.find("div", class_="cldt-price").find('h2').text)

In [7]:
price

6000

##### 3 features de présentation

In [18]:
basic_data = soup.find("div", class_='cldt-stage-basic-data')

In [20]:
km = extract_int(basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[0].string)
power_kW = extract_int(basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[2].string)
power_CH = extract_int(basic_data.find_all("span", class_="sc-font-m cldt-stage-primary-keyfact")[0].string)

In [21]:
km, power_kW, power_CH

(10600, 22, 30)

#### ===> Détails brut <===

##### Etat 

In [252]:
etat_template = pd.DataFrame({"État": "", 
                                  "Propriétaires préc.": [np.nan],
                                  "Dernier contrôle technique": [datetime.datetime(1970, 1, 1)],
                                  "Garantie": [datetime.datetime(1970, 1, 1)],
                                  "Carnet d'entretien": [np.nan]})

etat_template

,État,Propriétaires préc.,Dernier contrôle technique,Garantie,Carnet d'entretien
0,,NaN,1970-01-01,1970-01-01,NaN


In [179]:
#liste exhaustive des items de Etat
etat_features = ["État", "Propriétaires préc.", "Dernier contrôle technique", "Garantie", "Carnet d'entretien"]

etat_row = {"État": "", 
              "Propriétaires préc.": np.nan,
              "Dernier contrôle technique": datetime.datetime(1970, 1, 1),
              "Garantie": datetime.datetime(1970, 1, 1),
              "Carnet d'entretien": np.nan}

In [335]:
#scrapping de la branche contenant les key/values de Etat
etat = soup.find("div", class_="cldt-categorized-data cldt-data-section sc-pull-left")

In [292]:
#création liste des key et des valeurs => création dictionnaire de key, values
etat_key = [i.text for i in etat.find_all("dt")]
etat_value = [i.text.replace('\n', '') for i in etat.find_all("dd")]
etat_dict = {k: v for k,v in zip(etat_key, etat_value)}
etat_dict

{'État': 'Occasion',
 'Propriétaires préc.': '1',
 'Dernier contrôle technique': '07/2020',
 "Carnet d'entretien": ''}

In [302]:
#crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
for k_site in [i.text for i in etat.find_all("dt")]:
    for k_list in etat_features:
        if k_site == k_list:
            etat_row[k_list]=etat_dict[k_site]

#ajout du dictionnaire en tant que nouvelle ligne au df
etat_row

{'État': 'Occasion',
 'Propriétaires préc.': '1',
 'Dernier contrôle technique': '07/2020',
 'Garantie': datetime.datetime(1970, 1, 1, 0, 0),
 "Carnet d'entretien": ''}

In [257]:
#ajout du dictionnaire en tant que nouvelle ligne au df
etat_template = etat_template.append(etat_row, ignore_index=True)

In [258]:
etat_template

,État,Propriétaires préc.,Dernier contrôle technique,Garantie,Carnet d'entretien
0,,NaN,1970-01-01 00:00:00,1970-01-01,NaN
1,Occasion,1,07/2020,NaT,
2,NaN,NaN,NaN,NaT,
3,NaN,NaN,NaN,NaT,django


##### Caractéristiques

In [290]:
#liste exhaustive des items de caractéristiques
car_features = ["Marque", "N° d'annonce", "Année", "Couleur extérieure", "Type de peinture", "Couleur originale", "Carrosserie"]
car_row = {"Marque": "", "N° d'annonce": np.NaN, 
              "Année": [datetime.datetime(1970, 1, 1)], 
              "Couleur extérieure": "", 
              "Type de peinture": "", 
              "Couleur originale": "", 
              "Carrosserie": ""}

In [283]:
#scrapping de la branche contenant les key/values de caractéristiques
car = soup.find("div", class_="cldt-categorized-data cldt-data-section sc-pull-right")

In [287]:
#création liste des key et des valeurs => création dictionnaire de key, values
car_key = [i.text for i in car.find_all("dt")]
car_value = [i.text.replace('\n', '') for i in car.find_all("dd")]
car_dict = {k: v for k,v in zip(car_key, car_value)}

In [288]:
car_dict

{'Marque': 'Yamaha',
 'Année': '2018',
 'Couleur extérieure': 'Noir',
 'Carrosserie': 'Scooter'}

In [299]:
#crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
for k_site in [i.text for i in car.find_all("dt")]:
    for k_list in car_features:
        if k_site == k_list:
            car_row[k_list]=car_dict[k_site]

car_row

{'Marque': 'Yamaha',
 "N° d'annonce": nan,
 'Année': '2018',
 'Couleur extérieure': 'Noir',
 'Type de peinture': '',
 'Couleur originale': '',
 'Carrosserie': 'Scooter'}

##### Transmission

In [330]:
transm_features = ["Transmission", "Vitesses", "Cylindrée", "Cylindres", "Poids à vide"]
transm_row = {"Transmission": "", 
                  "Vitesses": np.nan, 
                  "Cylindrée": np.nan, 
                  "Cylindres": 0, 
                  "Poids à vide" : np.nan}

In [324]:
#scrapping de la branche contenant les key/values de caractéristiques
transm = soup.find_all("div", class_="cldt-categorized-data cldt-data-section sc-pull-left")[1]

In [326]:
#création liste des key et des valeurs => création dictionnaire de key, values
transm_key = [i.text for i in transm.find_all("dt")]
transm_value = [i.text.replace('\n', '') for i in transm.find_all("dd")]
transm_dict = {k: v for k,v in zip(transm_key, transm_value)}

In [327]:
transm_dict

{'Transmission': 'Yamaha', 'Cylindrée': '2018', 'Poids à vide': 'Noir'}

In [331]:
#crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
for k_site in [i.text for i in transm.find_all("dt")]:
    for k_list in transm_features:
        if k_site == k_list:
            transm_row[k_list]=transm_dict[k_site]

#ajout du dictionnaire en tant que nouvelle ligne au df
transm_row

{'Transmission': 'Yamaha',
 'Vitesses': nan,
 'Cylindrée': '2018',
 'Cylindres': 0,
 'Poids à vide': 'Noir'}

##### . . . Create dict with features . . .

In [ ]:
new_features
new_row
new_soup (soup)
row_name

In [334]:
def create_dict(new_features, new_row, soup, row_name):
    
    #création liste des key et des valeurs => création dictionnaire de key, values
    feature_key = [i.text for i in soup.find_all("dt")]
    feature_value = [i.text.replace('\n', '') for i in soup.find_all("dd")]
    feature_dict = {k: v for k,v in zip(feature_key, feature_value)}
    
    #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
    for k_site in [i.text for i in feature_key]:
        for k_list in new_features:
            if k_site == k_list:
                new_row[k_list]=feature_dict[k_site]
    
    row_name = new_row
    
    return row_name

In [333]:
new_features = transm_features
new_row = transm_row
soup = transm
row_name = transm_row

create_dict(new_features, new_row, soup, row_name)

AttributeError: 'str' object has no attribute 'text'

### Fonction de scrapping d'une annonce

#### Scrap description

In [983]:
url = 'https://www.autoscout24.fr/offres/yamaha-x-max-125-x-max-essence-noir-732462bf-74dc-44a6-81eb-42134d613141?cldtidx=8&cldtsrc=listPage&searchId=1361383082'
resp_single = requests.get(url)
soup_ = BeautifulSoup(resp_single.content, 'html.parser')

In [984]:
soup_.find_all("div", class_="sc-grid-col-6 sc-grid-col-s-12")[1].text

'\n\n()\n\n\nAfficher plus\nAfficher moins\n'

In [976]:
for edge in [soup_.find_all("div", class_="sc-grid-col-6 sc-grid-col-s-12")[1]]:
        print(edge.text)
        print('----------------')



Moto totalement modifiée sauf moteur et cadre. COMME NEUVE. Toujours dormi au garage.Poignées, bras oscillant, guidon et commandes avancées REBUFFINI.Jantes RICKS (devant 130 - derrière 250) / fourche SJPStage 1 fait par Harley Annecy - filtre à air Screaming Eagle - Réservoir et pare-boue faits main par un carrossier - trappes aviation - scelle avec gel faite sur mesure -phare etc...TOUTES LES PIECES AVEC CERTIFICATION TUV!!!Service fait du jour c/o Harley GenèveVendue pour cause déménagement. Contact: cif.jfc@bluewin.ch


Afficher plus
Afficher moins

----------------


#### fonction Etat : scrap_etat(soup)

In [336]:
def scrap_etat(soup):
    etat_features = ["État", "Propriétaires préc.", "Dernier contrôle technique", "Garantie", "Carnet d'entretien"]
    etat_row = {"État": "", 
              "Propriétaires préc.": np.nan,
              "Dernier contrôle technique": datetime.datetime(1970, 1, 1),
              "Garantie": datetime.datetime(1970, 1, 1),
              "Carnet d'entretien": np.nan}
    
    #scrapping de la branche contenant les key/values de Etat
    etat = soup.find("div", class_="cldt-categorized-data cldt-data-section sc-pull-left")
    
    #création liste des key et des valeurs => création dictionnaire de key, values
    etat_key = [i.text for i in etat.find_all("dt")]
    etat_value = [i.text.replace('\n', '') for i in etat.find_all("dd")]
    etat_dict = {k: v for k,v in zip(etat_key, etat_value)}
    
    #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
    for k_site in [i.text for i in etat.find_all("dt")]:
        for k_list in etat_features:
            if k_site == k_list:
                etat_row[k_list]=etat_dict[k_site]
    
    return etat_row

In [337]:
scrap_etat(soup_)

{'État': 'Occasion',
 'Propriétaires préc.': nan,
 'Dernier contrôle technique': '04/2020',
 'Garantie': '3 mois',
 "Carnet d'entretien": ''}

#### fonction caractéristiques : scrap_car(soup)

In [338]:
def scrap_car(soup):
    car_features = ["Marque", "N° d'annonce", "Année", "Couleur extérieure", "Type de peinture", "Couleur originale", "Carrosserie"]
    car_row = {"Marque": "", "N° d'annonce": np.NaN, 
              "Année": [datetime.datetime(1970, 1, 1)], 
              "Couleur extérieure": "", 
              "Type de peinture": "", 
              "Couleur originale": "", 
              "Carrosserie": ""}
    
    #scrapping de la branche contenant les key/values de caractéristiques
    car = soup.find("div", class_="cldt-categorized-data cldt-data-section sc-pull-right")
    
    #création liste des key et des valeurs => création dictionnaire de key, values
    car_key = [i.text for i in car.find_all("dt")]
    car_value = [i.text.replace('\n', '') for i in car.find_all("dd")]
    car_dict = {k: v for k,v in zip(car_key, car_value)}
    
    #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
    for k_site in [i.text for i in car.find_all("dt")]:
        for k_list in car_features:
            if k_site == k_list:
                car_row[k_list]=car_dict[k_site]
    
    return car_row

In [339]:
scrap_car(soup_)

{'Marque': 'Ducati',
 "N° d'annonce": nan,
 'Année': '2012',
 'Couleur extérieure': 'Rouge',
 'Type de peinture': 'Métallisé',
 'Couleur originale': 'gris',
 'Carrosserie': 'Roadster'}

#### fonction transmission : scrap_transm(soup)

In [340]:
def scrap_transm(soup):
    transm_features = ["Transmission", "Vitesses", "Cylindrée", "Cylindres", "Poids à vide"]
    transm_row = {"Transmission": "", 
                      "Vitesses": np.nan, 
                      "Cylindrée": np.nan, 
                      "Cylindres": 0, 
                      "Poids à vide" : np.nan}
    #scrapping de la branche contenant les key/values de caractéristiques
    transm = soup.find_all("div", class_="cldt-categorized-data cldt-data-section sc-pull-left")[1]
    
    #création liste des key et des valeurs => création dictionnaire de key, values
    transm_key = [i.text for i in transm.find_all("dt")]
    transm_value = [i.text.replace('\n', '') for i in transm.find_all("dd")]
    transm_dict = {k: v for k,v in zip(transm_key, transm_value)}
    
    #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
    for k_site in [i.text for i in transm.find_all("dt")]:
        for k_list in transm_features:
            if k_site == k_list:
                transm_row[k_list]=transm_dict[k_site]
    
    return transm_row    

In [341]:
scrap_transm(soup_)

{'Transmission': 'Boîte manuelle',
 'Vitesses': '5',
 'Cylindrée': '1.100 cm³',
 'Cylindres': '8',
 'Poids à vide': nan}

#### function : scrap all details

In [365]:
url = "https://www.autoscout24.fr/offres/piaggio-x-8-125-essence-noir-657c4002-1eb6-4b2c-ae78-b575350e7c6a?cldtidx=9&cldtsrc=listPage&searchId=-1532419452"

In [366]:
resp_single = requests.get(url)
soup_ = BeautifulSoup(resp_single.content, 'html.parser')
uniq_id = "as24_" + as24_announce_reference(soup_)

In [360]:
def scrap_part_one(soup_):
    part_one_row = {"url": "",
                      "reference": np.nan,
                      "model": "",
                      "price": np.nan,
                      "mileage": 0,
                      "power_kW": 0,
                      "power_CH": 0}
    uniq_id = "as24_" + as24_announce_reference(soup_)
    
    part_one_row["url"] = url
    part_one_row["reference"] = uniq_id
    part_one_row["model"] = soup_.find("div", class_="cldt-headline").find("span", class_="cldt-detail-makemodel sc-ellipsis").text
    part_one_row["price"] = extract_int(soup_.find("div", class_="cldt-price").find('h2').text)
    
    basic_data = soup_.find("div", class_='cldt-stage-basic-data')
    part_one_row["mileage"] = extract_int(basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[0].string)
    part_one_row["power_kW"] = extract_int(basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[2].string)
    part_one_row["power_CH"] = extract_int(basic_data.find_all("span", class_="sc-font-m cldt-stage-primary-keyfact")[0].string)
    
    return part_one_row

In [370]:
basic_data = soup_.find("div", class_='cldt-stage-basic-data')
basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[2].string

'- kW'

In [358]:
def scrap_all_details(url):
    resp_single = requests.get(url)
    soup_ = BeautifulSoup(resp_single.content, 'html.parser')
    
    
    part_one = scrap_part_one(soup_)
    etat = scrap_etat(soup_)
    car = scrap_car(soup_)
    transm = scrap_transm(soup_)
    
    dall = {}
    dall.update(part_one)
    dall.update(etat)
    dall.update(car)
    dall.update(transm)
    return dall

In [393]:
url = "https://www.autoscout24.fr/offres/piaggio-mp3-400-lt-essence-bleu-a03570dd-9554-48ab-913f-65cbafc9f659?cldtidx=9&cldtsrc=listPage&searchId=947976848"
scrap_all_details(url)

{'url': 'https://www.autoscout24.fr/offres/piaggio-mp3-400-lt-essence-bleu-a03570dd-9554-48ab-913f-65cbafc9f659?cldtidx=9&cldtsrc=listPage&searchId=947976848',
 'reference': 'as24_a03570dd-9554-48ab-913f-65cbafc9f659',
 'model': 'Piaggio MP3 400',
 'price': 2899,
 'mileage': 45800,
 'power_kW': nan,
 'power_CH': nan,
 'État': 'Occasion',
 'Propriétaires préc.': nan,
 'Dernier contrôle technique': datetime.datetime(1970, 1, 1, 0, 0),
 'Garantie': datetime.datetime(1970, 1, 1, 0, 0),
 "Carnet d'entretien": nan,
 'Marque': 'Piaggio',
 "N° d'annonce": '2048173',
 'Année': '2011',
 'Couleur extérieure': 'Bleu',
 'Type de peinture': '',
 'Couleur originale': 'Bleu',
 'Carrosserie': 'Scooter',
 'Transmission': '',
 'Vitesses': nan,
 'Cylindrée': '400 cm³',
 'Cylindres': 0,
 'Poids à vide': nan}

In [ ]:
#je suis en train de créer la fonction pour récupérer les premières données
#ensuite faudra ajouter la fonction du dessous pour récupérer toutes les données
#essayer sur plusieurs url

#scrapper les urls depuis le listing d'annonces et les stocker
#créer une boucle avec les url et la fonction pour scrapper les données


In [131]:
def get_one_announce(url):
    
    resp_single = requests.get(url, headers=headers, params=params)
    soup_single = BeautifulSoup(resp_single.content, 'html.parser')
    uniq_id = "as24_" + as24_announce_reference(soup_single)
    
    announce["url"] = url    
    announce["reference"] = uniq_id
    announce["model"] = soup_.find("div", class_="cldt-headline").find("span", class_="cldt-detail-makemodel sc-ellipsis").text
    announce["price"] = extract_int(soup_.find("div", class_="cldt-price").find('h2').text)
    
    get_images(soup_single, uniq_id)
    
    list_single = []
    
    

### Scrap announce urls

#### Scrapping de l'url des annonde d'un listing

In [824]:
url = "https://www.autoscout24.fr/lst-moto/?sort=standard&desc=0&ustate=N%2CU&size=10&page=2&cy=F&atype=B&fc=0&qry=&recommended_sorting_based_id=a03570dd-9554-48ab-913f-65cbafc9f659&"

In [825]:
#Combien d'annonces par page
10
#Quel est le div d'une annonce
# cl-list-element cl-list-element-gap
#Ou tourver le lien url
resp_single = requests.get(url)
soup_ = BeautifulSoup(resp_single.content, 'html.parser')

In [826]:
#extracton du code html
soup_.find_all("div", class_="cldt-summary-titles")[9].find_all('a', href=True)[0]['href']

'/offres/bmw-s-1000-xr-essence-rouge-63abeec2-96d6-498a-a4dc-d9f2357e4284'

In [504]:
for a in range(10):
    soup_link = soup_.find_all("div", class_="cldt-summary-titles")[a].find_all('a', href=True)[0]
    print('https://www.autoscout24.fr' + soup_link['href'])

https://www.autoscout24.fr/offres/triumph-bonneville-t120-t120-r-morgo-kit-750-cc-essence-noir-b04e6be2-14bb-495d-b64c-44e575e654a5
https://www.autoscout24.fr/offres/triumph-others-t100-r-daytona-essence-bleu-ee4624dc-c773-4f83-9dc6-99c4782ff761
https://www.autoscout24.fr/offres/triumph-others-t100-r-daytona-essence-argent-abd9b1f1-a4ee-4a85-8f31-2046f7f0428c
https://www.autoscout24.fr/offres/yamaha-tracer-900-gt-abs-2020-essence-rouge-7642b5a0-a394-4393-9557-8d67b866f333
https://www.autoscout24.fr/offres/yamaha-yfz-450-r-essence-bleu-66aec1cb-0643-4edb-b7ce-aa095f8c5c5d
https://www.autoscout24.fr/offres/bsa-a-65-sptifire-essence-rouge-078eb2c0-235d-437c-9bb3-6978fdce9a7d
https://www.autoscout24.fr/offres/bmw-s-1000-xr-essence-rouge-63abeec2-96d6-498a-a4dc-d9f2357e4284
https://www.autoscout24.fr/offres/bmw-c-evolution-electrique-599ed602-767c-45a0-b828-915dba58bcda
https://www.autoscout24.fr/offres/harley-davidson-softail-essence-2-temps-noir-cde41188-afcb-dc55-e053-e250040accb5
https:

In [713]:
def as24_url_generator(size, page):
    url_one = "www.autoscout24.fr/lst-moto/?sort=standard&desc=0&ustate=N%2CU&size="
    url_two = "&page="
    url_three = "&cy=F&atype=B&fc=10&qry=&recommended_sorting_based_id=b04e6be2-14bb-495d-b64c-44e575e654a5&"

#     url = url_one + str(size) + url_two + str(size) + url_three
    url = f"https://www.autoscout24.fr/lst-moto/?sort=standard&desc=0&ustate=N%2CU&size={size}&page={page}&cy=F&atype=B&fc=10&qry=&recommended_sorting_based_id=b04e6be2-14bb-495d-b64c-44e575e654a5&"
    return str(url)

In [716]:
as24_url_generator(20,2)

'https://www.autoscout24.fr/lst-moto/?sort=standard&desc=0&ustate=N%2CU&size=20&page=2&cy=F&atype=B&fc=10&qry=&recommended_sorting_based_id=b04e6be2-14bb-495d-b64c-44e575e654a5&'

#### Scrapping guid et prix du listing annonces

In [922]:
#lister les guid et prix correspondants à chaque annonce
def get_announces(soup_):
    announces_list = soup_.find_all("div", class_="cl-list-element cl-list-element-gap")
    for k in range(len(announces_list)):
        #get model
        model = announces_list[k].find("div", class_="cldt-summary-makemodelversion sc-ellipsis").find('h2').text
        announces_row['model']=model
        #get price
        price = announces_list[k].find("div", class_="cldt-summary-payment").find("span", class_="cldt-price sc-font-xl sc-font-bold").text
        announces_row['price']=price
        #get guid
        announces_row['guid']=announces_list[k]["data-guid"]
        #get url
        soup_link = soup_.find_all("div", class_="cldt-summary-titles")[k].find_all('a', href=True)[0]
        announces_row['url']='https://www.autoscout24.fr' + soup_link['href']

In [923]:
announces_list = soup_.find_all("div", class_="cl-list-element cl-list-element-gap")
for k in range(len(announces_list)):
    #get model
    model = announces_list[k].find("div", class_="cldt-summary-makemodelversion sc-ellipsis").find('h2').text
    print(model)
    #get price
    price = announces_list[k].find("div", class_="cldt-summary-payment").find("span", class_="cldt-price sc-font-xl sc-font-bold").text
    print(price)
    #get guid
    print(announces_list[k]["data-guid"])
    #get url
    soup_link = soup_.find_all("div", class_="cldt-summary-titles")[k].find_all('a', href=True)[0]
    print('https://www.autoscout24.fr' + soup_link['href'])

Velocette

€ 350,-

882f22c3-f496-8301-e053-0100007f5bc0
https://www.autoscout24.fr/offres/velocette-other-electrique-brun-882f22c3-f496-8301-e053-0100007f5bc0
Yamaha BWs

€ 400,-

a5252dc2-e0ff-4b0a-e053-0100007fd0f4
https://www.autoscout24.fr/offres/yamaha-bws-diesel-noir-a5252dc2-e0ff-4b0a-e053-0100007fd0f4
Peugeot Speedfight

€ 400,-

40080cc1-395d-4250-e053-0100007fb135
https://www.autoscout24.fr/offres/peugeot-speedfight-essence-blanc-40080cc1-395d-4250-e053-0100007fb135
MBK Booster

€ 400,-

1cb60f9d-d414-4a51-81f9-8a27d8a1dd16
https://www.autoscout24.fr/offres/mbk-booster-essence-noir-1cb60f9d-d414-4a51-81f9-8a27d8a1dd16
Vespa LX 50

€ 500,-

716725c3-b112-f23f-e053-0100007f719b
https://www.autoscout24.fr/offres/vespa-lx-50-essence-2-temps-bleu-716725c3-b112-f23f-e053-0100007f719b
Yamaha Aerox

€ 500,-

4ad337c1-beb4-427f-e053-0100007fb452
https://www.autoscout24.fr/offres/yamaha-aerox-essence-noir-4ad337c1-beb4-427f-e053-0100007fb452
Piaggio NRG Power

€ 500,-

f5fc0cc1-9739-2

#### composition du DataFrame

In [924]:
#lister les guid et prix correspondants à chaque annonce
#soup_d'une liste d'annonces

def get_announces(soup_):
    #Template of the dataframe with the urls of the several announces
    df_announces = pd.DataFrame({"model": "", 
                              "price": [np.nan],
                              "guid": "",
                              "url": ""})
    
    announces_row = {"model": "", 
                  "price": [np.nan],
                  "guid": "",
                  "url": ""}
    
    announces_list = soup_.find_all("div", class_="cl-list-element cl-list-element-gap")
    
    for k in range(len(announces_list)):
        #get model
        model = announces_list[k].find("div", class_="cldt-summary-makemodelversion sc-ellipsis").find('h2').text
        announces_row['model']=model
        #get price
        price = announces_list[k].find("div", class_="cldt-summary-payment").find("span", class_="cldt-price sc-font-xl sc-font-bold").text
        announces_row['price']=price
        #get guid
        announces_row['guid']=announces_list[k]["data-guid"]
        #get url
        soup_link = soup_.find_all("div", class_="cldt-summary-titles")[k].find_all('a', href=True)[0]
        announces_row['url']='https://www.autoscout24.fr' + soup_link['href']
        
        df_announces = df_announces.append(announces_row, ignore_index=True)
    
    df_announces = df_announces.drop(0)
    return df_announces

In [898]:
jack = get_announces(soup_)
jack

,model,price,guid,url
1,Triumph,"\n€ 11.500,-\n",abd9b1f1-a4ee-4a85-8f31-2046f7f0428c,https://www.autoscout24.fr/offres/triumph-othe...
2,Yamaha Tracer 900,"\n€ 12.066,-\n",7642b5a0-a394-4393-9557-8d67b866f333,https://www.autoscout24.fr/offres/yamaha-trace...
3,Yamaha YFZ 450,"\n€ 12.500,-\n",66aec1cb-0643-4edb-b7ce-aa095f8c5c5d,https://www.autoscout24.fr/offres/yamaha-yfz-4...
4,BSA A 65,"\n€ 14.500,-\n",078eb2c0-235d-437c-9bb3-6978fdce9a7d,https://www.autoscout24.fr/offres/bsa-a-65-spt...
5,BMW S 1000 XR,"\n€ 14.999,-\n",63abeec2-96d6-498a-a4dc-d9f2357e4284,https://www.autoscout24.fr/offres/bmw-s-1000-x...
6,BMW C Evolution,"\n€ 16.525,-\n",599ed602-767c-45a0-b828-915dba58bcda,https://www.autoscout24.fr/offres/bmw-c-evolut...
7,Harley-Davidson Softail,"\n€ 19.000,-\n",cde41188-afcb-dc55-e053-e250040accb5,https://www.autoscout24.fr/offres/harley-david...
8,Autres,"\n€ 19.800,-\n",082034e4-655e-4e03-863e-734200a01876,https://www.autoscout24.fr/offres/others-other...
9,Norton,"\n€ 20.900,-\n",2b97adbf-c230-4fe6-8e8e-0e035ae3c529,https://www.autoscout24.fr/offres/norton-other...
10,BMW R 1250 RT,"\n€ 21.900,-\n",66387b6b-2c8b-43f4-acf2-770dbfb0c4be,https://www.autoscout24.fr/offres/bmw-r-1250-r...


In [742]:
jack.iloc[9,3]

'https://www.autoscout24.fr/offres/others-others-climax-m1-bobber-essence-noir-082034e4-655e-4e03-863e-734200a01876'

#### Scrap des url de toutes les pages listing

In [779]:
#afficher les url de tous les listings d'annonces
def get_all_listing_url(nb_requests = 20):
    #création du dataframe
    df_listing = pd.DataFrame({"url": [np.nan]
                            })
    url_row = {"url": ""}

    for page in range(1,nb_requests+1):
        url_row['url'] = str(as24_url_generator(20, page))
        #ajouter le dictionnaire et le texte après la virgule dans parenthèse du append
        df_listing = df_listing.append(url_row, ignore_index=True)
    df_listing = df_listing.drop(0)
    return df_listing

In [794]:
get_all_listing_url(2)

,url
1,https://www.autoscout24.fr/lst-moto/?sort=stan...
2,https://www.autoscout24.fr/lst-moto/?sort=stan...


In [788]:
listing_list.iloc[0,0]

'https://www.autoscout24.fr/lst-moto/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=F&atype=B&fc=10&qry=&recommended_sorting_based_id=b04e6be2-14bb-495d-b64c-44e575e654a5&'

#### Scrap annonces de tous les listings

In [895]:
def get_all_announces(nb_of_pages):
    listing_list = get_all_listing_url(nb_of_pages)
    
    df_announces = pd.DataFrame({"model": "", 
                              "price": [np.nan],
                              "guid": "",
                              "url": ""})
    
    #boucle sur les url d'annonces extraites de la liste de listing
    for page in range(len(listing_list.iloc[:,0])):
        #sleep
        time.sleep(8)
        
        #extraction du script HTML du listing
        url = listing_list.iloc[page,0]
        resp_single = requests.get(url)
        soup_ = BeautifulSoup(resp_single.content, 'html.parser')
        
        new_df = get_announces(soup_)
        df_announces = pd.concat([df_announces, new_df])
    
    df_announces = df_announces.drop(0).reset_index(drop=True)
    return df_announces

In [929]:
df = get_all_announces(19)

In [934]:
df.head(30)

,model,price,guid,url
0,BMW R 1150 R Rockster,"\n€ 1.700,-\n",909652f8-01a9-4db9-af72-7be7205cfad9,https://www.autoscout24.fr/offres/bmw-r-1150-r...
1,Skyteam Dax 125,"\n€ 1.890,-\n",0f48fbac-52f4-202b-e053-0100007fe080,https://www.autoscout24.fr/offres/skyteam-dax-...
2,Yamaha XV 1100,"\n€ 2.700,-\n",ca7fb4c3-2028-7c4b-e053-0100007f16d2,https://www.autoscout24.fr/offres/yamaha-xv-11...
3,Vespa 150,"\n€ 3.450,-\n",ee13f4bf-ae32-4ca6-a549-57687080ce4c,https://www.autoscout24.fr/offres/vespa-150-s-...
4,BMW R 1100 GS,"\n€ 4.500,-\n",88cfe9c2-ac18-cc6a-e053-0100007f56ac,https://www.autoscout24.fr/offres/bmw-r-1100-g...
5,Yamaha XT 600,"\n€ 4.800,-\n",6688bc02-b3a4-4909-8ca2-c4f717f835f9,https://www.autoscout24.fr/offres/yamaha-xt-60...
6,Honda CB 1000,"\n€ 5.490,-\n",52e137d3-9f1b-49b9-816b-5b82fefa8fa9,https://www.autoscout24.fr/offres/honda-cb-100...
7,BMW R 1200 RT,"\n€ 5.888,-\n",db8e59ae-61b4-496a-9ae6-7ce4e5225475,https://www.autoscout24.fr/offres/bmw-r-1200-r...
8,BMW R 100 GS Paris-Dakar,"\n€ 7.800,-\n",a615a310-62ca-4641-a02e-657d673246a6,https://www.autoscout24.fr/offres/bmw-r-100-gs...
9,Ducati XDiavel,"\n€ 8.990,-\n",c9a6b7d5-939e-4be8-a45a-bc8adf43075c,https://www.autoscout24.fr/offres/ducati-xdiav...


In [935]:
df.iloc[21,3]

'https://www.autoscout24.fr/offres/others-others-climax-m1-bobber-essence-noir-082034e4-655e-4e03-863e-734200a01876'